## Conclusion from SQL and panda pre analysis
### Observation
* The code of a control area is unique. A control area will only have one unit and multiple SCPs but units may be duplicate among stations. Linename is messy. => Just use Control Area
* Time is not consistent too. => Use morning/afternoon/night...
* Get needed columns to save almost 100 mb
* Can keep desc-recovered but will need to clean duplicate data with both regular and recovered
* sql query may be faster
### Completed
* Query data from database
### To do
* datetime frame
* clean duplicate data with both regular and recovered
* Date and time analysis
* Substract entries and exits to get the numbers

In [1]:
#*****need to run*****
from sqlalchemy import create_engine
#from sqlalchemy import inspect
import pandas as pd

In [2]:
#*****need to run*****
engine = create_engine("sqlite:///mta.db")

In [3]:
#*****need to run*****
#df = pd.read_sql('SELECT ca, scp, station, date, time, desc, entries, exits FROM mta_data;', engine)
df = pd.read_sql('''SELECT ca, scp, station, date, time, desc, entries, exits FROM mta_data
                    WHERE date NOT IN ("02/27/2021", "02/28/2021", "06/01/2021", 
                                       "06/02/2021", "06/03/2021", "06/04/2021")
                    ORDER BY ca, date, time, scp, desc;''', engine)

In [4]:
#*****need to run*****
df["o_index"] = df.index
df["o_index"]

0                0
1                1
2                2
3                3
4                4
            ...   
2747752    2747752
2747753    2747753
2747754    2747754
2747755    2747755
2747756    2747756
Name: o_index, Length: 2747757, dtype: int64

In [45]:
#*****need to run*****
#drop duplicate data with wrong recover audit first
mask_dup = df[(df["o_index"] == 300706) |
          (df["o_index"] == 1089956) |
          (df["o_index"] == 1382146) |
          (df["o_index"] == 1382154) |
          ((df["CA"] == "N100") &
           (df["SCP"] == "00-00-00") &
           (df["ENTRIES"] == 818272) &
           (df["o_index"] != 842617))].index
df.drop(mask_dup, inplace= True)
#then, drop duplicate data with wrong regular data
df.drop_duplicates(subset=["CA", "SCP", "STATION", "DATE", "TIME"], keep= 'last', inplace=True)
df.drop(['o_index'], axis = 1)

In [56]:
#*****need to run*****
df["DATE_TIME"] = pd.to_datetime(df.DATE + " " + df.TIME, format="%m/%d/%Y %H:%M:%S")
df["DAY"] = df["DATE_TIME"].dt.weekday

In [58]:
print(df.head(3))
print(df.sample(3))
print(df.tail(3))

     CA       SCP STATION        DATE      TIME     DESC  ENTRIES    EXITS  \
0  A002  02-00-00   59 ST  03/01/2021  03:00:00  REGULAR  7537860  2570718   
1  A002  02-00-01   59 ST  03/01/2021  03:00:00  REGULAR  6677242  1516347   
2  A002  02-03-00   59 ST  03/01/2021  03:00:00  REGULAR  1416509  5346219   

   o_index           DATE_TIME  DAY  
0        0 2021-03-01 03:00:00    0  
1        1 2021-03-01 03:00:00    0  
2        2 2021-03-01 03:00:00    0  
            CA       SCP          STATION        DATE      TIME     DESC  \
591538    N001  01-06-01    INWOOD-207 ST  04/14/2021  01:00:00  REGULAR   
1476657  N601A  01-03-02  LEXINGTON AV/63  03/18/2021  16:00:00  REGULAR   
652327    N035  00-00-03   CATHEDRAL PKWY  04/22/2021  05:00:00  REGULAR   

           ENTRIES   EXITS  o_index           DATE_TIME  DAY  
591538   202168581  495658   591538 2021-04-14 01:00:00    2  
1476657    1351455  984058  1476657 2021-03-18 16:00:00    3  
652327      534533  423882   652327 2021-

In [48]:
dup = (df
       .groupby(["CA", "SCP", "STATION", "DATE", "TIME"])
       .ENTRIES.count()
       .reset_index()
       .sort_values("ENTRIES"))
dup[dup.ENTRIES > 1].shape

(49, 6)

In [41]:
df.iloc[1382154]

CA                   N526
SCP              02-00-04
STATION    DELANCEY/ESSEX
DATE           04/19/2021
TIME             01:00:00
DESC              REGULAR
ENTRIES             90488
EXITS               31301
o_index           1382158
Name: 1382158, dtype: object

In [50]:
dup = (df
       .groupby(["CA", "SCP", "STATION", "DATE", "TIME"])
       .ENTRIES.count()
       .reset_index()
       .sort_values("ENTRIES"))
dup[dup.ENTRIES > 1]

,CA,SCP,STATION,DATE,TIME,ENTRIES


In [52]:
mask = ((df["CA"] == "B024") & 
(df["SCP"] == "00-05-00") & 
(df["STATION"] == "KINGS HWY") &
((df["DATE"] == "05/07/2021") |
 (df["DATE"] == "05/08/2021") |
 (df["DATE"] == "05/09/2021")))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
232609,B024,00-05-00,KINGS HWY,05/07/2021,00:00:00,REGULAR,262152,0,232609
232614,B024,00-05-00,KINGS HWY,05/07/2021,04:00:00,REGULAR,262152,0,232614
232620,B024,00-05-00,KINGS HWY,05/07/2021,08:00:00,REGULAR,262152,0,232620
232626,B024,00-05-00,KINGS HWY,05/07/2021,12:00:00,REGULAR,262152,0,232626
232631,B024,00-05-00,KINGS HWY,05/07/2021,16:00:00,REGULAR,25,0,232631
232637,B024,00-05-00,KINGS HWY,05/07/2021,20:00:00,REGULAR,25,0,232637
232643,B024,00-05-00,KINGS HWY,05/08/2021,00:00:00,REGULAR,25,0,232643
232649,B024,00-05-00,KINGS HWY,05/08/2021,04:00:00,REGULAR,25,0,232649
232655,B024,00-05-00,KINGS HWY,05/08/2021,08:00:00,REGULAR,25,0,232655
232661,B024,00-05-00,KINGS HWY,05/08/2021,12:00:00,REGULAR,26,0,232661


In [7]:
print(df.head(3))
print(df.tail(3))

     CA       SCP STATION        DATE      TIME     DESC  ENTRIES    EXITS  \
0  A002  02-00-00   59 ST  03/01/2021  03:00:00  REGULAR  7537860  2570718   
1  A002  02-00-01   59 ST  03/01/2021  03:00:00  REGULAR  6677242  1516347   
2  A002  02-03-00   59 ST  03/01/2021  03:00:00  REGULAR  1416509  5346219   

   o_index  
0        0  
1        1  
2        2  
            CA       SCP        STATION        DATE      TIME     DESC  \
2747754  TRAM2  00-03-01  RIT-ROOSEVELT  05/31/2021  21:00:00  REGULAR   
2747755  TRAM2  00-05-00  RIT-ROOSEVELT  05/31/2021  21:00:00  REGULAR   
2747756  TRAM2  00-05-01  RIT-ROOSEVELT  05/31/2021  21:00:00  REGULAR   

         ENTRIES   EXITS  o_index  
2747754  3133585  397464  2747754  
2747755        0       0  2747755  
2747756     5554     577  2747756  


In [54]:
#save almost 80 mb
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2747682 entries, 0 to 2747756
Data columns (total 10 columns):
 #   Column     Dtype         
---  ------     -----         
 0   CA         object        
 1   SCP        object        
 2   STATION    object        
 3   DATE       object        
 4   TIME       object        
 5   DESC       object        
 6   ENTRIES    int64         
 7   EXITS      int64         
 8   o_index    int64         
 9   DATE_TIME  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 230.6+ MB


In [9]:
for col in df:
    print(col, df[col].nunique())

CA 750
SCP 224
STATION 379
DATE 92
TIME 56818
DESC 2
ENTRIES 1806455
EXITS 1717090
o_index 2747757


In [10]:
for col in df:
    print(df[col].value_counts())

PTH22    23551
R610     13798
PTH03    13540
PTH02    12173
R238     12130
         ...  
R177      1440
J022      1104
H037      1104
A085       552
OB01       313
Name: CA, Length: 750, dtype: int64
00-00-00    254844
00-00-01    253111
00-00-02    225012
00-00-03    113928
00-03-00     99554
             ...  
00-04-09       521
02-02-04       520
03-01-07       508
03-00-08         3
01-FF-02         1
Name: SCP, Length: 224, dtype: int64
34 ST-PENN STA     56370
FULTON ST          53013
GRD CNTRL-42 ST    49022
23 ST              40300
86 ST              36002
                   ...  
CHAUNCEY ST         1640
HARLEM 148 ST       1640
SUTTER AV           1104
CLEVELAND ST        1104
ORCHARD BEACH        313
Name: STATION, Length: 379, dtype: int64
04/12/2021    30321
03/19/2021    30167
03/16/2021    30147
04/07/2021    30094
03/15/2021    30073
              ...  
04/29/2021    29800
05/23/2021    29789
05/02/2021    29789
05/14/2021    29761
03/14/2021    27156
Name: DATE, Lengt

In [11]:
pd.read_sql('''SELECT a.ca, a.scp, a.station, a.date, a.time, a.entries, a.exits, b.date, b.time, b.entries, b.exits
               FROM mta_data a, mta_data b 
               WHERE a.ca = b.ca 
                 AND a.scp = b.scp
                 AND a.date = b.date
                 AND b.time > a.time
               LIMIT 15;''', engine)

,CA,SCP,STATION,DATE,TIME,ENTRIES,EXITS,DATE,TIME,ENTRIES,EXITS
0,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,07:00:00,7537260,2570445
1,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,11:00:00,7537285,2570500
2,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,15:00:00,7537354,2570546
3,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,19:00:00,7537501,2570582
4,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,23:00:00,7537589,2570596
5,A002,02-00-00,59 ST,02/27/2021,07:00:00,7537260,2570445,02/27/2021,11:00:00,7537285,2570500
6,A002,02-00-00,59 ST,02/27/2021,07:00:00,7537260,2570445,02/27/2021,15:00:00,7537354,2570546
7,A002,02-00-00,59 ST,02/27/2021,07:00:00,7537260,2570445,02/27/2021,19:00:00,7537501,2570582
8,A002,02-00-00,59 ST,02/27/2021,07:00:00,7537260,2570445,02/27/2021,23:00:00,7537589,2570596
9,A002,02-00-00,59 ST,02/27/2021,11:00:00,7537285,2570500,02/27/2021,15:00:00,7537354,2570546


In [12]:
# doesn't work for overnight
pd.read_sql('''SELECT * 
               FROM (SELECT a.ca, a.scp, a.station, a.date, a.time, a.entries, a.exits, b.date, b.time, b.entries, b.exits,
                            ROW_NUMBER() Over (Partition By a.ca, a.scp, a.date, a.time Order By a.time) SORT
                     FROM mta_data a, mta_data b 
                     WHERE a.ca = b.ca 
                       AND a.scp = b.scp
                       AND a.date = b.date
                       AND b.time > a.time
                     LIMIT 30)
               WHERE sort = 1;''', engine)

,ca,scp,station,date,time,entries,exits,date:1,time:1,entries:1,exits:1,SORT
0,A002,02-00-00,59 ST,02/27/2021,03:00:00,7537255,2570434,02/27/2021,07:00:00,7537260,2570445,1
1,A002,02-00-00,59 ST,02/27/2021,07:00:00,7537260,2570445,02/27/2021,11:00:00,7537285,2570500,1
2,A002,02-00-00,59 ST,02/27/2021,11:00:00,7537285,2570500,02/27/2021,15:00:00,7537354,2570546,1
3,A002,02-00-00,59 ST,02/27/2021,15:00:00,7537354,2570546,02/27/2021,19:00:00,7537501,2570582,1
4,A002,02-00-00,59 ST,02/27/2021,19:00:00,7537501,2570582,02/27/2021,23:00:00,7537589,2570596,1
5,A002,02-00-00,59 ST,02/28/2021,03:00:00,7537598,2570598,02/28/2021,07:00:00,7537602,2570602,1
6,A002,02-00-00,59 ST,02/28/2021,07:00:00,7537602,2570602,02/28/2021,11:00:00,7537625,2570637,1
7,A002,02-00-00,59 ST,02/28/2021,11:00:00,7537625,2570637,02/28/2021,15:00:00,7537696,2570671,1
8,A002,02-00-00,59 ST,02/28/2021,15:00:00,7537696,2570671,02/28/2021,19:00:00,7537791,2570709,1
9,A002,02-00-00,59 ST,02/28/2021,19:00:00,7537791,2570709,02/28/2021,23:00:00,7537852,2570717,1


In [13]:
dup = (df
       .groupby(["CA", "SCP", "STATION", "DATE", "TIME"])
       .ENTRIES.count()
       .reset_index()
       .sort_values("ENTRIES", ascending=False))
dup[dup.ENTRIES > 1]

,CA,SCP,STATION,DATE,TIME,ENTRIES
232922,B024,00-05-00,KINGS HWY,05/04/2021,16:00:00,2
232914,B024,00-05-00,KINGS HWY,05/03/2021,08:00:00,2
2402733,R417,00-03-01,PARKCHESTER,03/26/2021,05:00:00,2
1127520,N324,00-06-03,JKSN HT-ROOSVLT,04/14/2021,00:00:00,2
2313732,R314,00-00-02,PROSPECT AV,05/28/2021,05:00:00,2
...,...,...,...,...,...,...
232918,B024,00-05-00,KINGS HWY,05/04/2021,00:00:00,2
232921,B024,00-05-00,KINGS HWY,05/04/2021,12:00:00,2
232920,B024,00-05-00,KINGS HWY,05/04/2021,08:00:00,2
232929,B024,00-05-00,KINGS HWY,05/05/2021,20:00:00,2


In [14]:
#example1
mask = ((df["CA"] == "B024") & 
(df["SCP"] == "00-05-00") & 
(df["STATION"] == "KINGS HWY") &
(df["DATE"] == "05/04/2021") &
((df["TIME"] == "04:00:00") | 
 (df["TIME"] == "08:00:00") | 
 (df["TIME"] == "12:00:00")))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
232508,B024,00-05-00,KINGS HWY,05/04/2021,04:00:00,REGULAR,262152,0,232508
232513,B024,00-05-00,KINGS HWY,05/04/2021,08:00:00,RECOVR AUD,17,0,232513
232514,B024,00-05-00,KINGS HWY,05/04/2021,08:00:00,REGULAR,262152,0,232514
232519,B024,00-05-00,KINGS HWY,05/04/2021,12:00:00,RECOVR AUD,18,0,232519
232520,B024,00-05-00,KINGS HWY,05/04/2021,12:00:00,REGULAR,262152,0,232520


In [15]:
#example2
#is recover audit always the wrong one? -> no
#N071 is not
mask = ((df["CA"] == "N071") & 
(df["SCP"] == "00-00-00") & 
(df["STATION"] == "34 ST-PENN STA") &
(df["DATE"] == "04/08/2021") &
((df["TIME"] == "04:00:00") | 
 (df["TIME"] == "08:00:00") | 
 (df["TIME"] == "12:00:00")))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
744738,N071,00-00-00,34 ST-PENN STA,04/08/2021,04:00:00,REGULAR,6336196,17886776,744738
744774,N071,00-00-00,34 ST-PENN STA,04/08/2021,08:00:00,RECOVR AUD,6336210,17886963,744774
744775,N071,00-00-00,34 ST-PENN STA,04/08/2021,08:00:00,REGULAR,241178,732169,744775
744786,N071,00-00-00,34 ST-PENN STA,04/08/2021,12:00:00,RECOVR AUD,6336244,17887217,744786


In [16]:
#sort by different way
#B024!
dup = (df
       .groupby(["CA", "SCP", "STATION", "DATE", "TIME"])
       .ENTRIES.count()
       .reset_index()
       .sort_values(["CA","SCP","DATE","TIME"]))
dup[dup.ENTRIES > 1]

,CA,SCP,STATION,DATE,TIME,ENTRIES
232900,B024,00-05-00,KINGS HWY,05/01/2021,00:00:00,2
232902,B024,00-05-00,KINGS HWY,05/01/2021,08:00:00,2
232903,B024,00-05-00,KINGS HWY,05/01/2021,12:00:00,2
232904,B024,00-05-00,KINGS HWY,05/01/2021,16:00:00,2
232905,B024,00-05-00,KINGS HWY,05/01/2021,20:00:00,2
...,...,...,...,...,...,...
1380867,N526,02-00-01,DELANCEY/ESSEX,04/18/2021,21:00:00,2
1380868,N526,02-00-01,DELANCEY/ESSEX,04/19/2021,01:00:00,2
2313732,R314,00-00-02,PROSPECT AV,05/28/2021,05:00:00,2
2399973,R417,00-00-01,PARKCHESTER,03/26/2021,05:00:00,2


In [17]:
#B204
dup = (df
       .groupby(["CA", "SCP", "STATION", "DATE", "TIME"])
       .ENTRIES.count()
       .reset_index()
       .sort_values(["CA","SCP","DATE","TIME"]))
print(len(dup[(dup.ENTRIES > 1) & (dup.CA == "B024")]))
dup[(dup.ENTRIES > 1) & (dup.CA == "B024")]

35


,CA,SCP,STATION,DATE,TIME,ENTRIES
232900,B024,00-05-00,KINGS HWY,05/01/2021,00:00:00,2
232902,B024,00-05-00,KINGS HWY,05/01/2021,08:00:00,2
232903,B024,00-05-00,KINGS HWY,05/01/2021,12:00:00,2
232904,B024,00-05-00,KINGS HWY,05/01/2021,16:00:00,2
232905,B024,00-05-00,KINGS HWY,05/01/2021,20:00:00,2
232906,B024,00-05-00,KINGS HWY,05/02/2021,00:00:00,2
232907,B024,00-05-00,KINGS HWY,05/02/2021,04:00:00,2
232908,B024,00-05-00,KINGS HWY,05/02/2021,08:00:00,2
232909,B024,00-05-00,KINGS HWY,05/02/2021,12:00:00,2
232910,B024,00-05-00,KINGS HWY,05/02/2021,16:00:00,2


In [18]:
# so...regular of B204 is the wrong one
mask = ((df["CA"] == "B024") & 
(df["SCP"] == "00-05-00") & 
(df["STATION"] == "KINGS HWY") &
((df["DATE"] == "05/07/2021") |
 (df["DATE"] == "05/08/2021")))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
232608,B024,00-05-00,KINGS HWY,05/07/2021,00:00:00,RECOVR AUD,22,0,232608
232609,B024,00-05-00,KINGS HWY,05/07/2021,00:00:00,REGULAR,262152,0,232609
232614,B024,00-05-00,KINGS HWY,05/07/2021,04:00:00,REGULAR,262152,0,232614
232619,B024,00-05-00,KINGS HWY,05/07/2021,08:00:00,RECOVR AUD,22,0,232619
232620,B024,00-05-00,KINGS HWY,05/07/2021,08:00:00,REGULAR,262152,0,232620
232625,B024,00-05-00,KINGS HWY,05/07/2021,12:00:00,RECOVR AUD,23,0,232625
232626,B024,00-05-00,KINGS HWY,05/07/2021,12:00:00,REGULAR,262152,0,232626
232631,B024,00-05-00,KINGS HWY,05/07/2021,16:00:00,REGULAR,25,0,232631
232637,B024,00-05-00,KINGS HWY,05/07/2021,20:00:00,REGULAR,25,0,232637
232643,B024,00-05-00,KINGS HWY,05/08/2021,00:00:00,REGULAR,25,0,232643


In [19]:
#non B204?
dup = (df
       .groupby(["CA", "SCP", "STATION", "DATE", "TIME"])
       .ENTRIES.count()
       .reset_index()
       .sort_values(["CA","SCP","DATE","TIME"]))
print(len(dup[(dup.ENTRIES > 1) & (dup.CA != "B024")]))
dup[(dup.ENTRIES > 1) & (dup.CA != "B024")]

39


,CA,SCP,STATION,DATE,TIME,ENTRIES
300520,C018,00-00-04,36 ST,05/18/2021,09:00:00,2
743565,N071,00-00-00,34 ST-PENN STA,04/08/2021,08:00:00,2
744119,N071,00-00-01,34 ST-PENN STA,04/08/2021,08:00:00,2
744673,N071,00-00-02,34 ST-PENN STA,04/08/2021,08:00:00,2
745227,N071,00-00-03,34 ST-PENN STA,04/08/2021,08:00:00,2
745781,N071,00-00-04,34 ST-PENN STA,04/08/2021,08:00:00,2
746359,N071,00-06-00,34 ST-PENN STA,04/08/2021,08:00:00,2
842503,N100,00-00-00,HIGH ST,03/06/2021,03:00:00,2
842504,N100,00-00-00,HIGH ST,03/06/2021,07:00:00,2
842505,N100,00-00-00,HIGH ST,03/06/2021,11:00:00,2


In [20]:
#C018 => wrong recover audit on EXITS
mask = ((df["CA"] == "C018") & 
(df["SCP"] == "00-00-04") & 
(df["STATION"] == "36 ST") &
(df["DATE"] == "05/18/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
300694,C018,00-00-04,36 ST,05/18/2021,01:00:00,REGULAR,13017615,6034408,300694
300700,C018,00-00-04,36 ST,05/18/2021,05:00:00,REGULAR,13017621,6034411,300700
300706,C018,00-00-04,36 ST,05/18/2021,09:00:00,RECOVR AUD,13017811,5510261,300706
300707,C018,00-00-04,36 ST,05/18/2021,09:00:00,REGULAR,13017811,6034545,300707
300713,C018,00-00-04,36 ST,05/18/2021,13:00:00,REGULAR,13017958,6034643,300713
300719,C018,00-00-04,36 ST,05/18/2021,17:00:00,REGULAR,13018168,6034810,300719
300725,C018,00-00-04,36 ST,05/18/2021,21:00:00,REGULAR,13018377,6034933,300725


In [21]:
#N071 at 8 am on 4/8/2021 => wrong regular
mask = ((df["CA"] == "N071") &  
(df["STATION"] == "34 ST-PENN STA") &
(df["DATE"] == "04/08/2021") &
((df["TIME"] == "04:00:00") | 
 (df["TIME"] == "08:00:00") | 
 (df["TIME"] == "12:00:00")))

df[mask].sort_values(["CA","SCP","DATE","TIME"])

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
744738,N071,00-00-00,34 ST-PENN STA,04/08/2021,04:00:00,REGULAR,6336196,17886776,744738
744774,N071,00-00-00,34 ST-PENN STA,04/08/2021,08:00:00,RECOVR AUD,6336210,17886963,744774
744775,N071,00-00-00,34 ST-PENN STA,04/08/2021,08:00:00,REGULAR,241178,732169,744775
744786,N071,00-00-00,34 ST-PENN STA,04/08/2021,12:00:00,RECOVR AUD,6336244,17887217,744786
744739,N071,00-00-01,34 ST-PENN STA,04/08/2021,04:00:00,REGULAR,3493957,7767231,744739
744776,N071,00-00-01,34 ST-PENN STA,04/08/2021,08:00:00,RECOVR AUD,3493963,7767298,744776
744777,N071,00-00-01,34 ST-PENN STA,04/08/2021,08:00:00,REGULAR,2587793,2626240,744777
744787,N071,00-00-01,34 ST-PENN STA,04/08/2021,12:00:00,RECOVR AUD,3493984,7767435,744787
744740,N071,00-00-02,34 ST-PENN STA,04/08/2021,04:00:00,REGULAR,22267,56239,744740
744778,N071,00-00-02,34 ST-PENN STA,04/08/2021,08:00:00,RECOVR AUD,22269,56308,744778


In [43]:
#wrong recover audit
#3/10/2021?????
mask = ((df["CA"] == "N100") & 
(df["SCP"] == "00-00-00") & 
(df["STATION"] == "HIGH ST") &
((df["DATE"] == "03/05/2021") | 
 (df["DATE"] == "03/06/2021") | 
 (df["DATE"] == "03/07/2021") | 
 (df["DATE"] == "03/08/2021") |
 (df["DATE"] == "03/09/2021") | 
 (df["DATE"] == "03/10/2021") | 
 (df["DATE"] == "03/11/2021")))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
842587,N100,00-00-00,HIGH ST,03/05/2021,03:00:00,REGULAR,816253,1726988,842587
842590,N100,00-00-00,HIGH ST,03/05/2021,07:00:00,REGULAR,816265,1727075,842590
842593,N100,00-00-00,HIGH ST,03/05/2021,11:00:00,REGULAR,816321,1727413,842593
842596,N100,00-00-00,HIGH ST,03/05/2021,15:00:00,REGULAR,816415,1727624,842596
842599,N100,00-00-00,HIGH ST,03/05/2021,19:00:00,REGULAR,816619,1727910,842599
842602,N100,00-00-00,HIGH ST,03/05/2021,23:00:00,REGULAR,816695,1728027,842602
842605,N100,00-00-00,HIGH ST,03/06/2021,03:00:00,RECOVR AUD,818272,1731687,842605
842606,N100,00-00-00,HIGH ST,03/06/2021,03:00:00,REGULAR,816719,1728050,842606
842609,N100,00-00-00,HIGH ST,03/06/2021,07:00:00,RECOVR AUD,818272,1731687,842609
842610,N100,00-00-00,HIGH ST,03/06/2021,07:00:00,REGULAR,816722,1728063,842610


In [24]:
#doesn't matter
mask = ((df["CA"] == "N114") & 
(df["SCP"] == "01-00-01") & 
(df["STATION"] == "FRANKLIN AV") &
(df["DATE"] == "03/10/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
878257,N114,01-00-01,FRANKLIN AV,03/10/2021,00:00:00,REGULAR,627492,783379,878257
878263,N114,01-00-01,FRANKLIN AV,03/10/2021,04:00:00,REGULAR,627493,783400,878263
878269,N114,01-00-01,FRANKLIN AV,03/10/2021,08:00:00,RECOVR AUD,627514,783473,878269
878270,N114,01-00-01,FRANKLIN AV,03/10/2021,08:00:00,REGULAR,627514,783474,878270
878276,N114,01-00-01,FRANKLIN AV,03/10/2021,12:00:00,REGULAR,627544,783614,878276
878282,N114,01-00-01,FRANKLIN AV,03/10/2021,16:00:00,REGULAR,627582,783756,878282
878288,N114,01-00-01,FRANKLIN AV,03/10/2021,20:00:00,REGULAR,627643,783982,878288


In [25]:
#wrong recover audit
mask = ((df["CA"] == "N310") & 
(df["SCP"] == "01-06-00") & 
(df["STATION"] == "QUEENS PLAZA") &
(df["DATE"] == "05/20/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
1089936,N310,01-06-00,QUEENS PLAZA,05/20/2021,01:00:00,REGULAR,927819,335762,1089936
1089941,N310,01-06-00,QUEENS PLAZA,05/20/2021,05:00:00,REGULAR,927820,335767,1089941
1089946,N310,01-06-00,QUEENS PLAZA,05/20/2021,09:00:00,REGULAR,927870,335916,1089946
1089951,N310,01-06-00,QUEENS PLAZA,05/20/2021,13:00:00,REGULAR,927919,335964,1089951
1089956,N310,01-06-00,QUEENS PLAZA,05/20/2021,17:00:00,RECOVR AUD,917544,17105542,1089956
1089957,N310,01-06-00,QUEENS PLAZA,05/20/2021,17:00:00,REGULAR,928040,336006,1089957
1089962,N310,01-06-00,QUEENS PLAZA,05/20/2021,21:00:00,REGULAR,928085,336043,1089962


In [26]:
#doesn't matter
mask = ((df["CA"] == "N324") & 
(df["SCP"] == "00-06-03") & 
(df["STATION"] == "JKSN HT-ROOSVLT") &
(df["DATE"] == "04/14/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
1123295,N324,00-06-03,JKSN HT-ROOSVLT,04/14/2021,00:00:00,RECOVR AUD,8391213,2290172,1123295
1123296,N324,00-06-03,JKSN HT-ROOSVLT,04/14/2021,00:00:00,REGULAR,8391213,2290173,1123296
1123312,N324,00-06-03,JKSN HT-ROOSVLT,04/14/2021,04:00:00,REGULAR,8391216,2290204,1123312
1123328,N324,00-06-03,JKSN HT-ROOSVLT,04/14/2021,08:00:00,REGULAR,8391502,2290232,1123328
1123344,N324,00-06-03,JKSN HT-ROOSVLT,04/14/2021,12:00:00,REGULAR,8391972,2290289,1123344
1123360,N324,00-06-03,JKSN HT-ROOSVLT,04/14/2021,16:00:00,REGULAR,8392237,2290408,1123360
1123376,N324,00-06-03,JKSN HT-ROOSVLT,04/14/2021,20:00:00,REGULAR,8392474,2290595,1123376


In [27]:
#wrong regular
mask = ((df["CA"] == "N329") & 
(df["SCP"] == "00-03-02") & 
(df["STATION"] == "WOODHAVEN BLVD") &
(df["DATE"] == "05/08/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
1143962,N329,00-03-02,WOODHAVEN BLVD,05/08/2021,01:00:00,RECOVR AUD,12538859,12056519,1143962
1143963,N329,00-03-02,WOODHAVEN BLVD,05/08/2021,01:00:00,REGULAR,4825276,5534671,1143963
1143967,N329,00-03-02,WOODHAVEN BLVD,05/08/2021,05:00:00,RECOVR AUD,12538860,12056522,1143967
1143968,N329,00-03-02,WOODHAVEN BLVD,05/08/2021,05:00:00,REGULAR,4825280,5534700,1143968
1143972,N329,00-03-02,WOODHAVEN BLVD,05/08/2021,09:00:00,RECOVR AUD,12538900,12056543,1143972
1143973,N329,00-03-02,WOODHAVEN BLVD,05/08/2021,09:00:00,REGULAR,4825375,5534785,1143973
1143979,N329,00-03-02,WOODHAVEN BLVD,05/08/2021,13:00:00,REGULAR,12538962,12056653,1143979
1143985,N329,00-03-02,WOODHAVEN BLVD,05/08/2021,17:00:00,REGULAR,12539078,12056863,1143985
1143992,N329,00-03-02,WOODHAVEN BLVD,05/08/2021,21:00:00,REGULAR,12539160,12056994,1143992


In [28]:
#wrong recover audit
mask = ((df["CA"] == "N526") & 
(df["SCP"] == "02-00-01") & 
(df["STATION"] == "DELANCEY/ESSEX") &
((df["DATE"] == "04/18/2021") | 
 (df["DATE"] == "04/19/2021")))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
1382111,N526,02-00-01,DELANCEY/ESSEX,04/18/2021,01:00:00,REGULAR,3714266,2906413,1382111
1382118,N526,02-00-01,DELANCEY/ESSEX,04/18/2021,05:00:00,REGULAR,3714273,2906423,1382118
1382125,N526,02-00-01,DELANCEY/ESSEX,04/18/2021,09:00:00,REGULAR,3714297,2906464,1382125
1382132,N526,02-00-01,DELANCEY/ESSEX,04/18/2021,13:00:00,REGULAR,3714362,2906576,1382132
1382139,N526,02-00-01,DELANCEY/ESSEX,04/18/2021,17:00:00,REGULAR,3714479,2906741,1382139
1382146,N526,02-00-01,DELANCEY/ESSEX,04/18/2021,21:00:00,RECOVR AUD,0,0,1382146
1382147,N526,02-00-01,DELANCEY/ESSEX,04/18/2021,21:00:00,REGULAR,3714622,2906859,1382147
1382154,N526,02-00-01,DELANCEY/ESSEX,04/19/2021,01:00:00,RECOVR AUD,0,0,1382154
1382155,N526,02-00-01,DELANCEY/ESSEX,04/19/2021,01:00:00,REGULAR,3714726,2906881,1382155
1382162,N526,02-00-01,DELANCEY/ESSEX,04/19/2021,05:00:00,REGULAR,3714727,2906883,1382162


In [29]:
#doesn't matter
mask = ((df["CA"] == "R314") & 
(df["SCP"] == "00-00-02") & 
(df["STATION"] == "PROSPECT AV") &
(df["DATE"] == "05/28/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
2313756,R314,00-00-02,PROSPECT AV,05/28/2021,01:00:00,REGULAR,67258358,16839155,2313756
2313759,R314,00-00-02,PROSPECT AV,05/28/2021,05:00:00,RECOVR AUD,67258361,16839157,2313759
2313760,R314,00-00-02,PROSPECT AV,05/28/2021,05:00:00,REGULAR,67258362,16839157,2313760
2313763,R314,00-00-02,PROSPECT AV,05/28/2021,09:00:00,REGULAR,67258495,16839196,2313763
2313766,R314,00-00-02,PROSPECT AV,05/28/2021,13:00:00,REGULAR,67258573,16839232,2313766
2313769,R314,00-00-02,PROSPECT AV,05/28/2021,17:00:00,REGULAR,67258707,16839293,2313769
2313772,R314,00-00-02,PROSPECT AV,05/28/2021,21:00:00,REGULAR,67258771,16839330,2313772


In [30]:
#doesn't matter
mask = ((df["CA"] == "R417") & 
(df["SCP"] == "00-00-01") & 
(df["STATION"] == "PARKCHESTER") &
(df["DATE"] == "03/26/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
2401141,R417,00-00-01,PARKCHESTER,03/26/2021,01:00:00,REGULAR,835659,105861,2401141
2401153,R417,00-00-01,PARKCHESTER,03/26/2021,05:00:00,RECOVR AUD,835697,105861,2401153
2401154,R417,00-00-01,PARKCHESTER,03/26/2021,05:00:00,REGULAR,835698,105861,2401154
2401167,R417,00-00-01,PARKCHESTER,03/26/2021,09:00:00,REGULAR,836260,105874,2401167
2401179,R417,00-00-01,PARKCHESTER,03/26/2021,13:00:00,REGULAR,836569,105892,2401179
2401191,R417,00-00-01,PARKCHESTER,03/26/2021,17:00:00,REGULAR,836910,105937,2401191
2401203,R417,00-00-01,PARKCHESTER,03/26/2021,21:00:00,REGULAR,837066,105989,2401203


In [31]:
#doesn't matter
mask = ((df["CA"] == "R417") & 
(df["SCP"] == "00-03-01") & 
(df["STATION"] == "PARKCHESTER") &
(df["DATE"] == "03/26/2021"))

df[mask]

,CA,SCP,STATION,DATE,TIME,DESC,ENTRIES,EXITS,o_index
2401146,R417,00-03-01,PARKCHESTER,03/26/2021,01:00:00,REGULAR,1241990,7007535,2401146
2401159,R417,00-03-01,PARKCHESTER,03/26/2021,05:00:00,RECOVR AUD,1241994,7007548,2401159
2401160,R417,00-03-01,PARKCHESTER,03/26/2021,05:00:00,REGULAR,1241994,7007549,2401160
2401172,R417,00-03-01,PARKCHESTER,03/26/2021,09:00:00,REGULAR,1242027,7007635,2401172
2401184,R417,00-03-01,PARKCHESTER,03/26/2021,13:00:00,REGULAR,1242060,7007723,2401184
2401196,R417,00-03-01,PARKCHESTER,03/26/2021,17:00:00,REGULAR,1242100,7008001,2401196
2401208,R417,00-03-01,PARKCHESTER,03/26/2021,21:00:00,REGULAR,1242110,7008362,2401208


wrong:
* recovered: 25
* regular: 44
* doesn't matter: 5

what to do?